<a href="https://colab.research.google.com/github/jonnymagalhaesicev/pos_ia_modulo1/blob/main/Atividade_Modulo_1_Jonny_e_Sebastiao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Trabalho Módulo 1 Pós IA iCEV
#  Dupla
*  Jonny Magalhães
*  Sebastião Galeno

#  Desafio
#  Construir uma aplicação em Python que, ao receber a lista de compras, consiga:
*  1. Detectar compras duplicadas
*  2. Listar valores acima do limite (R$1000)
*  3. Identificar compras fora do horário comercial (antes de 08:00 ou depois de 18:00)
*  4. Organizar as compras por servidor usando dicionários
*  5. Exibir relatório com severidade da infração
*  6. Usar funções para modularização
*  7. Usar list comprehensions sempre que possível

In [2]:
import pandas as pd

In [122]:
def detectar_compras_duplicadas(compras, subset=None):
    df_duplicado = compras.duplicated(subset=subset)
    num_duplicados = df_duplicado.sum()
    if subset is None:
        print("Número de linhas duplicadas:", num_duplicados)
    else:
      print(f"Número de linhas duplicadas por {subset}:", num_duplicados)

def verificar_valores_suspeitos(compras, valor_suspeito=1000.0):
    return compras[compras['valor_em_real'] > valor_suspeito]

def compras_fora_de_horario(row):
  if row['data_da_compra'].hour < 8 or row['data_da_compra'].hour > 17:
    return 'Sim'
  return 'Não'

def classificar_horario_compra(compras):
    compras['CompraForaHorarioComercial'] = compras.apply(compras_fora_de_horario, axis=1)
    return compras

def organizar_por_servidor(compras):
    #Conversão forçada do tipo de item comprado
    compras['item_comprado'] = compras['item_comprado'].astype(str)

    #Agrupamento de compras por servidor
    df_compras_por_servidor = compras.groupby('nome_do_funcionario').agg(
        quantidade=('valor_em_real', 'count'),
        valor_total=('valor_em_real', 'sum'),
        itens_comprados=('item_comprado', lambda x: ', '.join(x))
    ).reset_index()


    return df_compras_por_servidor

def severidade(row):
  if row['quantidade'] < 5 or row['valor_total'] < 5000:
    return 'Baixa'
  elif row['quantidade'] < 10 or row['valor_total'] < 50000:
    return 'Média'
  else:
    return 'Alta'

def compra_suspeita(row, qtd_compras, valor_compra):
  if row['quantidade']> qtd_compras or row['valor_total'] > valor_compra:
    return 'Sim'
  return 'Não'

def gerar_relatorio(compras):
  compras['Severidade'] = compras.apply(severidade, axis=1)
  compras['Compra suspeita'] = compras.apply(lambda row: compra_suspeita(row, qtd_compras, valor_compra), axis=1)
  return compras

In [108]:
df = pd.read_csv('/content/public_servant_purchases_new.csv')
pd.set_option('display.float_format', '{:.2f}'.format)

# Questão 1. Detectar compras duplicadas

In [83]:
#Quantidade de linhas no dataset
df.shape

(100000, 4)

In [109]:
#Verificação de linhas completamente duplicadas
detectar_compras_duplicadas(df)

Número de linhas duplicadas: 0


Aqui percebe-se que não existem linhas completamente duplicadas

In [116]:
detectar_compras_duplicadas(df, ['item_comprado', 'data_da_compra'])

df['data_da_compra'] = pd.to_datetime(df['data_da_compra'])

df['data_da_compra_sem_hora'] = df['data_da_compra'].dt.date

detectar_compras_duplicadas(df, ['item_comprado', 'data_da_compra_sem_hora'])

Número de linhas duplicadas por ['item_comprado', 'data_da_compra']: 2
Número de linhas duplicadas por ['item_comprado', 'data_da_compra_sem_hora']: 85


Filtrando pelo item e data da compra temos 2 casos, no código abaixo removeremos os duplicados mantendo a primeira ocorrencia e deixaremos saparados os valores anteriores em df_duplicado

Para compras na mesma data ignorando a hora temos 85 registros duplicados.

In [86]:
#Código opcional caso desejemos remover linhas duplicadas.
#df.drop_duplicates(subset=['item_comprado', 'data_da_compra'], keep='first', inplace=True)

# Questão 2. Listar valores acima do limite (R$1000)

In [ ]:
valor_suspeito = float(input('Informe o valor que consideras suspeito '))
df_suspeito =  verificar_valores_suspeitos(df, valor_suspeito)
print(f'Abaixo temos um resumo dos valores suspeitos acima de {valor_suspeito}')
print(f'Foram encontrados {df_suspeito.shape[0]} compras acima de {valor_suspeito}')
df_suspeito.head()

# Questão 3. Identificar compras fora do horário comercial (antes de 08:00 ou depois de 18:00)

In [124]:
df = classificar_horario_compra(df)
df[df['CompraForaHorarioComercial'] =='Sim']

,nome_do_funcionario,item_comprado,valor_em_real,data_da_compra,data_da_compra_sem_hora,CompraForaHorarioComercial
0,Amanda Jones,Copy Paper,227.00,2024-12-01 18:05:04,2024-12-01,Sim
1,Patrick Kavuma,Counselling Services,88.00,2024-01-18 18:26:17,2024-01-18,Sim
5,PRITI SHARMA,Furniture,260.00,2024-03-24 01:10:00,2024-03-24,Sim
6,Dennis S.,Tapetes,13.00,2024-06-21 19:17:51,2024-06-21,Sim
8,Davinia A,Congress,196.85,2024-09-25 02:27:23,2024-09-25,Sim
...,...,...,...,...,...,...
4983,Jennifer Hernandez,Food,24.60,2024-09-16 06:40:03,2024-09-16,Sim
4991,Simon,Carteira 2014/2018,182.00,2024-08-01 07:06:48,2024-08-01,Sim
4993,Terry O\'Connor,PANCHO\'S MEXICANA,12.99,2024-05-31 19:50:17,2024-05-31,Sim
4995,Mrs. D Nelson,Purchase Order,20.21,2024-07-02 21:51:20,2024-07-02,Sim


# Questão 4. Organizar as compras por servidor usando dicionários

In [ ]:
#Agrupando compras por servidor
df_por_servidor = organizar_por_servidor(df)
df_por_servidor

In [ ]:
qtd_compras = int(input('Informe a quantidade de compras que deseja filtrar por servidor: '))
#Exibido compras por servidor com quantidade maior que o valor fornecido
df_compras_por_qtd = df_por_servidor[df_por_servidor['quantidade'] > qtd_compras]
df_compras_por_qtd

In [ ]:
valor_compra = int(input('Informe o valor de compras que deseja filtrar por servidor: '))
#Exibido compras por servidor com quantidade maior que o valor fornecido
df_compras_por_valor = df_por_servidor[df_por_servidor['valor_total'] > valor_compra]
df_compras_por_valor

# Questão 5. Exibir relatório com severidade da infração


In [106]:
#Exibindo relatório de compras por servidor onde a quantidade de compras ou o valor total sejam maiores que os fornecidos.
df_por_servidor = gerar_relatorio(df_por_servidor)
df_por_servidor

,nome_do_funcionario,quantidade,valor_total,itens_comprados,Severidade,Compra suspeita
0,"""BENJAMIN JAMES""",1,2.40,caderneta,Baixa,Não
1,"""Benedict Marien""",1,9.50,lsd,Baixa,Não
2,"""CHUA, MAY LING""",1,15.22,2x cocoa with dates 400g,Baixa,Não
3,"""Mrs. Harika D'Souza""",1,4.00,mate,Baixa,Não
4,"""Natalia Laura""",1,2.60,hambúrguer,Baixa,Não
...,...,...,...,...,...,...
85158,황용준,1,33.99,물통,Baixa,Não
85159,황우비,1,12.50,반려동물 맞춤 고기,Baixa,Não
85160,︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎...,1,0.00,cupom de desconto,Baixa,Não
85161,🔴Kevin Michael,1,129.99,mouse,Baixa,Não


In [107]:
#Aqui identificamos compras consideradas suspeitas pelo valor informado, mas que pela classificação do sistema tem severidade Baixa ou Média
#Estratégia para mitigar falso negativo por compras do servidor de acordo com a Severidade
df_por_servidor[(df_por_servidor['Compra suspeita'] == 'Sim') & (df_por_servidor['Severidade'] != 'Alta')]
df_por_servidor

,nome_do_funcionario,quantidade,valor_total,itens_comprados,Severidade,Compra suspeita
0,"""BENJAMIN JAMES""",1,2.40,caderneta,Baixa,Não
1,"""Benedict Marien""",1,9.50,lsd,Baixa,Não
2,"""CHUA, MAY LING""",1,15.22,2x cocoa with dates 400g,Baixa,Não
3,"""Mrs. Harika D'Souza""",1,4.00,mate,Baixa,Não
4,"""Natalia Laura""",1,2.60,hambúrguer,Baixa,Não
...,...,...,...,...,...,...
85158,황용준,1,33.99,물통,Baixa,Não
85159,황우비,1,12.50,반려동물 맞춤 고기,Baixa,Não
85160,︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎︎...,1,0.00,cupom de desconto,Baixa,Não
85161,🔴Kevin Michael,1,129.99,mouse,Baixa,Não
